# Representation Fairness
As we see in (past experiment), the dumb CNN with the Quadratic activation fails to learn a linear labelling (add plot). We find a minimal CNN architecture that does better and measure its sample complexity.

In [1]:
from IPython import get_ipython
from IPython.core.magic import register_cell_magic
ipython = get_ipython()

# New cell magic to use python variables in cell magic
@register_cell_magic
def format(line, cell):
    magic, program = line.split(' ')
    ipython.run_cell_magic(magic, program, cell.format(**globals()))

In [2]:
# Experiment variables
max_epochs      = 100 # Maximum number of training epochs per split
norm            = 2 # Labelling function parameter
accuracy        = 0.8 # Goal accuracy for convergence
proportion      = 0.5 # Proportion of FCNN parameters.
delete_results  = True # Delete results of previous runs for that parameter set (all models).
filepath        = 'results/week4/p'+str(norm)+'_acc'+str(accuracy)+'_e'+str(max_epochs)+'.pkl' # Results file

In [ ]:
%%format script powershell
python dim_vs_tr_size.py -actv "Quad" -arch "ParamFairCNN" -prop {proportion} -min 4 -max 6 -f {filepath} -e {max_epochs} -p {norm} -acc {accuracy} -del {delete_results}